In [ ]:
import requests

contests = requests.get("https://codeforces.com/api/contest.list").json()

In [ ]:
contests

In [ ]:
import requests

subs = requests.get("https://codeforces.com/api/contest.status?contestId=1500&from=1&count=10000").json()

In [ ]:
subs["result"][2]

In [ ]:
import requests

probs = requests.get("https://codeforces.com/api/problemset.problems")
print(probs)

In [ ]:
from lxml import html

tree = html.fromstring(req.text)

In [ ]:
tree.xpath('//*[@id="pageContent"]/div[3]/pre/text()')[0]

In [1]:
import os, sys
dr = os.path.abspath('cf_search/')
sys.path.append(dr)

from query.api import execute_query, get_default_adapter


adapter = get_default_adapter()

In [ ]:
query = """
{
    Contest {
        id 
            @filter(op_name: ">=", value: ["$min_id"]) 
            @filter(op_name: "<=", value: ["$max_id"]) 
            @output(out_name: "contest_id")
        name @output(out_name: "contest_name")
        
        out_Contest_Submission {
            id @output(out_name: "submission_id")
            programming_language 
                @filter(op_name: "=", value: ["$lang_name"])
                @output(out_name: "language")
            verdict 
                @filter(op_name: "=", value: ["$verdict"])
                @output(out_name: "verdict")
                
            out_Submission_Source {
                source_code @output(out_name: "source_code")
            }
            
            in_Submission_Problem {
                name @output(out_name: "problem_name")
                rating 
                    @filter(op_name: ">=", value: ["$min_rating"])
                    @output(out_name: "rating")
                solved_count @output(out_name: "solved_count")
            }
        }
    }
}
"""

args = {
    "min_id": 1497,
    "max_id": 1497,
    "min_rating": 1600,
    "lang_name": "Haskell",
    "verdict": "OK",
}

res = list(execute_query(adapter, query, args))
res

In [25]:
query = """
{
    Contest {
        id 
            @filter(op_name: ">=", value: ["$min_id"]) 
            @filter(op_name: "<=", value: ["$max_id"]) 
            @output(out_name: "contest_id")
        name @output(out_name: "contest_name")
        
        out_Contest_Submission {
            id @output(out_name: "submission_id")
            programming_language 
                @filter(op_name: "=", value: ["$lang_name"])
                @output(out_name: "language")
            verdict 
                @filter(op_name: "=", value: ["$verdict"])
                @output(out_name: "verdict")
                
            out_Submission_Source {
                out_Source_Children {
                    out_Node_Children @recurse(depth: 3) {
                        type @filter(op_name: "in_collection", value: ["$node_types"])
                        content
                            @filter(op_name: "has_substring", value: ["$node_content_str"])
                            @output(out_name: "node_content")
                    }
                }
            }
        }
    }
}
"""

args = {
    "min_id": 1497,
    "max_id": 1497,
    "lang_name": "Haskell",
    "verdict": "OK",
    "node_types": ['qualified_module'],
    "node_types": ['qualified_module'],
}

res = list(execute_query(adapter, query, args))
res

<Node kind=pragma, start_point=(0, 0), end_point=(0, 29)> 0 29 <class 'int'> <class 'int'>
<Node kind=ERROR, start_point=(2, 0), end_point=(56, 9)> 31 1250 <class 'int'> <class 'int'>
<Node kind=top_splice, start_point=(56, 10), end_point=(66, 15)> 1251 1608 <class 'int'> <class 'int'>
<Node kind=import, start_point=(2, 0), end_point=(2, 20)> 31 51 <class 'int'> <class 'int'>
<Node kind=import, start_point=(3, 0), end_point=(3, 25)> 52 77 <class 'int'> <class 'int'>
<Node kind=import, start_point=(4, 0), end_point=(4, 20)> 78 98 <class 'int'> <class 'int'>
<Node kind=import, start_point=(5, 0), end_point=(5, 24)> 99 123 <class 'int'> <class 'int'>
<Node kind=import, start_point=(6, 0), end_point=(6, 43)> 124 167 <class 'int'> <class 'int'>
<Node kind=import, start_point=(7, 0), end_point=(7, 16)> 168 184 <class 'int'> <class 'int'>
<Node kind=import, start_point=(8, 0), end_point=(8, 25)> 185 210 <class 'int'> <class 'int'>
<Node kind=comment, start_point=(8, 25), end_point=(9, 21)> 21

<Node kind="as", start_point=(3, 39), end_point=(3, 41)> 91 93 <class 'int'> <class 'int'>
<Node kind=module, start_point=(3, 42), end_point=(3, 43)> 94 95 <class 'int'> <class 'int'>
<Node kind=module, start_point=(3, 17), end_point=(3, 21)> 69 73 <class 'int'> <class 'int'>
<Node kind=module, start_point=(3, 22), end_point=(3, 32)> 74 84 <class 'int'> <class 'int'>
<Node kind=module, start_point=(3, 33), end_point=(3, 38)> 85 90 <class 'int'> <class 'int'>
<Node kind="import", start_point=(4, 0), end_point=(4, 6)> 96 102 <class 'int'> <class 'int'>
<Node kind=qualified_module, start_point=(4, 7), end_point=(4, 16)> 103 112 <class 'int'> <class 'int'>
<Node kind=module, start_point=(4, 7), end_point=(4, 11)> 103 107 <class 'int'> <class 'int'>
<Node kind=module, start_point=(4, 12), end_point=(4, 16)> 108 112 <class 'int'> <class 'int'>
<Node kind="import", start_point=(5, 0), end_point=(5, 6)> 113 119 <class 'int'> <class 'int'>
<Node kind=qualified_module, start_point=(5, 7), end_po

<Node kind=module, start_point=(3, 12), end_point=(3, 16)> 64 68 <class 'int'> <class 'int'>
<Node kind="import", start_point=(4, 0), end_point=(4, 6)> 69 75 <class 'int'> <class 'int'>
<Node kind=qualified_module, start_point=(4, 7), end_point=(4, 16)> 76 85 <class 'int'> <class 'int'>
<Node kind=import_list, start_point=(4, 17), end_point=(4, 23)> 86 92 <class 'int'> <class 'int'>
<Node kind=module, start_point=(4, 7), end_point=(4, 11)> 76 80 <class 'int'> <class 'int'>
<Node kind=module, start_point=(4, 12), end_point=(4, 16)> 81 85 <class 'int'> <class 'int'>
<Node kind="(", start_point=(4, 17), end_point=(4, 18)> 86 87 <class 'int'> <class 'int'>
<Node kind=import_item, start_point=(4, 18), end_point=(4, 22)> 87 91 <class 'int'> <class 'int'>
<Node kind=")", start_point=(4, 22), end_point=(4, 23)> 91 92 <class 'int'> <class 'int'>
<Node kind="import", start_point=(5, 0), end_point=(5, 6)> 93 99 <class 'int'> <class 'int'>
<Node kind="qualified", start_point=(5, 7), end_point=(5, 

<Node kind=signature, start_point=(17, 0), end_point=(17, 38)> 291 329 <class 'int'> <class 'int'>
<Node kind=ERROR, start_point=(17, 38), end_point=(17, 39)> 329 330 <class 'int'> <class 'int'>
<Node kind=function, start_point=(18, 0), end_point=(18, 17)> 331 348 <class 'int'> <class 'int'>
<Node kind=ERROR, start_point=(18, 17), end_point=(18, 18)> 348 349 <class 'int'> <class 'int'>
<Node kind=function, start_point=(19, 0), end_point=(19, 24)> 350 374 <class 'int'> <class 'int'>
<Node kind=ERROR, start_point=(19, 24), end_point=(19, 25)> 374 375 <class 'int'> <class 'int'>
<Node kind=function, start_point=(20, 0), end_point=(26, 0)> 376 585 <class 'int'> <class 'int'>
<Node kind=signature, start_point=(26, 0), end_point=(26, 26)> 585 611 <class 'int'> <class 'int'>
<Node kind=ERROR, start_point=(26, 26), end_point=(26, 27)> 611 612 <class 'int'> <class 'int'>
<Node kind=function, start_point=(27, 0), end_point=(33, 0)> 613 785 <class 'int'> <class 'int'>
<Node kind=signature, start_

<Node kind="module", start_point=(0, 0), end_point=(0, 6)> 0 6 <class 'int'> <class 'int'>
<Node kind=module, start_point=(0, 7), end_point=(0, 11)> 7 11 <class 'int'> <class 'int'>
<Node kind=where, start_point=(0, 12), end_point=(0, 17)> 12 17 <class 'int'> <class 'int'>
<Node kind=import, start_point=(2, 0), end_point=(2, 24)> 19 43 <class 'int'> <class 'int'>
<Node kind=import, start_point=(3, 0), end_point=(3, 34)> 44 78 <class 'int'> <class 'int'>
<Node kind=signature, start_point=(4, 0), end_point=(4, 13)> 79 92 <class 'int'> <class 'int'>
<Node kind=function, start_point=(5, 0), end_point=(9, 0)> 93 140 <class 'int'> <class 'int'>
<Node kind=signature, start_point=(9, 0), end_point=(9, 14)> 140 154 <class 'int'> <class 'int'>
<Node kind=function, start_point=(10, 0), end_point=(15, 0)> 155 268 <class 'int'> <class 'int'>
<Node kind=signature, start_point=(15, 0), end_point=(15, 25)> 268 293 <class 'int'> <class 'int'>
<Node kind=function, start_point=(16, 0), end_point=(21, 0)>

<Node kind=import, start_point=(0, 0), end_point=(0, 48)> 0 48 <class 'int'> <class 'int'>
<Node kind=import, start_point=(1, 0), end_point=(1, 45)> 49 94 <class 'int'> <class 'int'>
<Node kind=import, start_point=(2, 0), end_point=(2, 16)> 95 111 <class 'int'> <class 'int'>
<Node kind=import, start_point=(4, 0), end_point=(4, 20)> 113 133 <class 'int'> <class 'int'>
<Node kind=import, start_point=(5, 0), end_point=(5, 32)> 134 166 <class 'int'> <class 'int'>
<Node kind=import, start_point=(6, 0), end_point=(6, 32)> 167 199 <class 'int'> <class 'int'>
<Node kind=import, start_point=(7, 0), end_point=(7, 16)> 200 216 <class 'int'> <class 'int'>
<Node kind=import, start_point=(8, 0), end_point=(8, 21)> 217 238 <class 'int'> <class 'int'>
<Node kind=signature, start_point=(10, 0), end_point=(10, 21)> 240 261 <class 'int'> <class 'int'>
<Node kind=function, start_point=(11, 0), end_point=(14, 45)> 262 404 <class 'int'> <class 'int'>
<Node kind=signature, start_point=(16, 0), end_point=(16,

<Node kind=function, start_point=(27, 0), end_point=(34, 0)> 636 787 <class 'int'> <class 'int'>
<Node kind=type_alias, start_point=(34, 0), end_point=(34, 33)> 787 820 <class 'int'> <class 'int'>
<Node kind=signature, start_point=(36, 0), end_point=(36, 28)> 822 850 <class 'int'> <class 'int'>
<Node kind=function, start_point=(37, 0), end_point=(42, 0)> 851 955 <class 'int'> <class 'int'>
<Node kind=signature, start_point=(42, 0), end_point=(42, 21)> 955 976 <class 'int'> <class 'int'>
<Node kind=function, start_point=(43, 0), end_point=(47, 0)> 977 1074 <class 'int'> <class 'int'>
<Node kind=signature, start_point=(47, 0), end_point=(47, 26)> 1074 1100 <class 'int'> <class 'int'>
<Node kind=function, start_point=(48, 0), end_point=(51, 44)> 1101 1253 <class 'int'> <class 'int'>
<Node kind="import", start_point=(0, 0), end_point=(0, 6)> 0 6 <class 'int'> <class 'int'>
<Node kind="qualified", start_point=(0, 7), end_point=(0, 16)> 7 16 <class 'int'> <class 'int'>
<Node kind=qualified_m

[{'contest_id': 1497,
  'contest_name': 'Codeforces Round #708 (Div. 2)',
  'submission_id': 110946701,
  'language': 'Haskell',
  'verdict': 'OK',
  'node_content': 'Control.Monad'},
 {'contest_id': 1497,
  'contest_name': 'Codeforces Round #708 (Div. 2)',
  'submission_id': 110946701,
  'language': 'Haskell',
  'verdict': 'OK',
  'node_content': 'Data.Array.Unboxed'},
 {'contest_id': 1497,
  'contest_name': 'Codeforces Round #708 (Div. 2)',
  'submission_id': 110946701,
  'language': 'Haskell',
  'verdict': 'OK',
  'node_content': 'Data.Array.ST'},
 {'contest_id': 1497,
  'contest_name': 'Codeforces Round #708 (Div. 2)',
  'submission_id': 110946701,
  'language': 'Haskell',
  'verdict': 'OK',
  'node_content': 'Data.Array.MArray'},
 {'contest_id': 1497,
  'contest_name': 'Codeforces Round #708 (Div. 2)',
  'submission_id': 110946701,
  'language': 'Haskell',
  'verdict': 'OK',
  'node_content': 'Data.ByteString.Char8'},
 {'contest_id': 1497,
  'contest_name': 'Codeforces Round #708 

In [ ]:
query = """
{
    Contest {
        id 
            @filter(op_name: ">=", value: ["$min_id"]) 
            @filter(op_name: "<=", value: ["$max_id"]) 
            @output(out_name: "contest_id")
        name @output(out_name: "contest_name")
        
        out_Contest_Submission {
            id @output(out_name: "submission_id")
            programming_language 
                @output(out_name: "language")
            verdict 
                @filter(op_name: "=", value: ["$verdict"])
                @output(out_name: "verdict")
        }
    }
}
"""

args = {
    "min_id": 1497,
    "max_id": 1497,
    "min_rating": 1600,
    "verdict": "OK",
}

res = list(execute_query(adapter, query, args))
res

langs = set(map(lambda x: x['language'], res))
print(langs)

In [ ]:
query = """
{
    Problem {
        name @output(out_name: "name")
        solved_count
            @filter(op_name: "<=", value: ["$max_solves"])
            @output(out_name: "solved_count")
    }
}
"""

args = {
    "max_solves": 50,
}

res = list(execute_query(adapter, query, args))
res

In [4]:
from tree_sitter import Language, Parser
import os

Language.build_library(
  # Store the library in the `build` directory
  'build/languages.so',

  # Include one or more languages
  [
    'vendor/tree-sitter-cpp',
    'vendor/tree-sitter-haskell',
    'vendor/tree-sitter-python'
  ]
)

False

In [5]:
HASK_LANGUAGE = Language('build/languages.so', 'haskell')
parser = Parser()
parser.set_language(HASK_LANGUAGE)

hask_src = """
{-# LANGUAGE BangPatterns #-}

import Control.Monad
import qualified Data.ByteString.Char8 as B
import Data.Char
import Data.List (foldl')
-- import Debug.Trace
import Text.Printf

import Data.Maybe (fromJust)
import System.IO
import Text.Read

getNum :: IO Int
getNum = fst . fromJust . B.readInt <$> B.getLine

getNums :: IO [Int]
getNums = map ((fst . fromJust) . B.readInt) . B.split ' ' <$> B.getLine

showNums :: [Int] -> IO ()
showNums xs = putStrLn $ unwords $ map show xs

main :: IO ()
main = do
  t <- getNum
  forM_ [1 .. t] handleCase

handleCase :: Int -> IO ()
handleCase ti = do
  [n, c] <- getNums
  putStrLn $ printf "Case #%d: %s" ti (solve n c)

solve :: Int -> Int -> String
solve n c
  | c < (n - 1) || (c >= n * (n + 1) `div` 2) = "IMPOSSIBLE"
  | otherwise = construct [1..n] n c (n - 2)

construct :: [Int] -> Int -> Int -> Int -> String
construct as _ c (-1)
  | c > 0 = "IMPOSSIBLE"
  | otherwise = unwords $ map show as

construct as n c i  = construct as' n (c - stepCost) (i - 1)
  where
    stepCost = min (n - i) (c - i)
    j = i + stepCost - 1
    (p1, p1') = splitAt i as
    (p2, p3) = splitAt stepCost p1'
    as' = p1 ++ reverse p2 ++ p3

"""
hask_tree = parser.parse(bytes(hask_src, "utf8"))

In [ ]:
parser = Parser()
parser.set_language(Language('build/languages.so', 'cpp'))

cpp_src = """
#include <algorithm>
#include <iostream>
#include <numeric>
#include <vector>

using namespace std;

#define LL long long

int t, x, y;
string s;

void solve() {
  int l, r, u, d;
  l = r = u = d = 0;
  for (int i = 0; i < s.length(); i++) {
    switch (s[i]) {
    case 'R':
      r++; break;
    case 'L':
      l++;
      break;
    case 'U':
      u++;
      break;
    case 'D':
      d++;
      break;
    }
  }

  if (x > 0 && r < x) {
    cout << "NO" << endl;
    return;
  }
  if (x < 0 && l < (-x)) {
    cout << "NO" << endl;
    return;
  }

  if (y > 0 && u < y) {
    cout << "NO" << endl;
    return;
  }
  if (y < 0 && d < (-y)) {
    cout << "NO" << endl;
    return;
  }

    cout << "YES" << endl;
}

int main() {
  cin >> t;

  for (int c = 0; c < t; c++) {
    cin >> x >> y;
    cin >> s;

    solve();
  }

  return 0;
}

"""

cpp_tree = parser.parse(bytes(cpp_src, "utf8"))

In [20]:
hask_tree.root_node.children[2].children
#print(hask_tree.root_node.children[1].children[0].walk().current_field_name())
#dir(hask_tree.root_node.children[3].children[1])

[<Node kind="import", start_point=(4, 0), end_point=(4, 6)>,
 <Node kind="qualified", start_point=(4, 7), end_point=(4, 16)>,
 <Node kind=qualified_module, start_point=(4, 17), end_point=(4, 38)>,
 <Node kind="as", start_point=(4, 39), end_point=(4, 41)>,
 <Node kind=module, start_point=(4, 42), end_point=(4, 43)>]

In [ ]:
cpp_tree.root_node.children[0].children

In [ ]:
node = hask_tree.root_node.children[4]
s = node.start_byte
e = node.end_byte
hask_src[s:e]

In [ ]:
print(dir(tree.root_node.sexp))
tree.root_node.sexp().format()